In [1]:
import earthaccess
import numpy as np
from datetime import datetime, timedelta
import requests
import tempfile
import matplotlib.pyplot as plt
import rioxarray as rxr
import opera_utils
import pandas as pd
import os
import psutil
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import warnings

ncpus = len(psutil.Process().cpu_affinity())    # number of available CPUs
print('number of available CPUs: ', ncpus)
ncpus = 10  # manually set nworkers

number of available CPUs:  128


In [2]:
# Authenticate with NASA Earthdata Login
auth = earthaccess.login(strategy="netrc")
req_session = auth.get_session()

# Define the date range
start_date = datetime(2016, 7, 1)
end_date = datetime(2024, 9, 30)

threshold_snowcover = 40    # threshold of snow cover

# Define the CMR-STAC API endpoint and collection ID
stac_api_url = "https://cmr.earthdata.nasa.gov/stac/NSIDC_ECS"
collection_id = "C2015398723-NSIDC_ECS"  # This is the collection ID for MOD10CM

figure_dir = 'figures'
os.makedirs(figure_dir, exist_ok=True)

output_geojson = 'blockout_snow_disp_s1.geojson'

In [3]:
def get_snow_cover_data_filter(year, month, gdf_frame, threshold_snowcover=40):
    # Format the date string
    date_str = f"{year}-{month:02d}-01"
    end_date_str = (datetime(year, month, 1) + timedelta(days=32)).replace(day=1).strftime("%Y-%m-%d")
    
    # Search for granules
    results = earthaccess.search_data(
        short_name="MOD10CM",
        cloud_hosted=True,
        temporal=(date_str, end_date_str),
        bounding_box=(-180, -90, 180, 90)  # Global coverage
    )
    
    if results:
        # Get the first granule (should only be one per month)
        granule = results[0]
        urls = granule.data_links()
        
        if urls:
            url = urls[0]
            print(f"Streaming snow cover data for {year}-{month:02d}...")

            try:
                # Stream the content to a temporary file
                with tempfile.NamedTemporaryFile(delete=False, suffix='.hdf') as temp_file:
                    print(f"Downloading file from {url}")
                    with requests.get(url, stream=True) as response:
                        response.raise_for_status()  # Raise an exception for bad status codes
                        for chunk in response.iter_content(chunk_size=8192):
                            temp_file.write(chunk)
                    temp_file_path = temp_file.name
                
                print(f"File downloaded successfully to temporary location: {temp_file_path}")

                fname=f"HDF4_EOS:EOS_GRID:{temp_file_path}:MOD_CMG_Snow_5km:Snow_Cover_Monthly_CMG"
                dataset = rxr.open_rasterio(fname)
                
                snowcover = dataset.isel(band=0) # Select the first band and plot it
                gdf_frame = gdf_frame.to_crs(snowcover.rio.crs)

                # Clip the raster to the GeoDataFrame
                clipped = snowcover.rio.clip(gdf_frame.geometry)
                clipped = clipped.where(clipped <= 250, np.nan)

                # Plot the clipped raster
                plt.figure(figsize=(10, 6))
                clipped.plot(cmap="viridis", vmin=0, vmax=100)
                plt.xlim(-180,-50)
                plt.ylim(0,80)
                plt.title(f"Monthly Snow Cover {dataset.attrs['RANGEBEGINNINGDATE']} to {dataset.attrs['RANGEENDINGDATE']}")
                figname = f"{figure_dir}/monthly_snow_cover_{dataset.attrs['RANGEBEGINNINGDATE']}_{dataset.attrs['RANGEENDINGDATE']}.png"
                plt.savefig(figname, dpi=300, bbox_inches='tight')
                plt.close()

                # Calculate mean snow cover for each frame_id
                mean_values = []

                # Suppress RuntimeWarnings temporarily
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore", category=RuntimeWarning)
                    
                    for idx, row in gdf_frame.iterrows():
                        frame_id = row['frame_id']
                        geometry = row['geometry']
                        
                        # Clip the raster to the current geometry
                        clipped = snowcover.rio.clip([geometry])
                        
                        # Remove pixels above 250
                        clipped = clipped.where(clipped <= 249, np.nan)
                        
                        # Calculate the mean of the clipped raster 
                        mean_value = np.nanmean(clipped).item()  # .item() extracts the scalar from the array
                        mean_values.append(mean_value)

                # Append mean_values as a column in gdf_frames
                gdf_frame["mean_snowcover"] = mean_values

                # Replace nans to -1 so we can track later
                if hasattr(gdf_frame['mean_snowcover'], 'fillna'):
                    gdf_frame['mean_snowcover'] = gdf_frame['mean_snowcover'].fillna(-1)
                else:
                    gdf_frame['mean_snowcover'].fillna(-1, inplace=True)

                gdf_frame['to_process'] = gdf_frame['mean_snowcover'].apply(lambda x: 0 if x >= threshold_snowcover else 1)

                gdf_frame = gdf_frame.drop(['is_land', 'is_north_america', 'orbit_pass', 'mean_snowcover'], axis=1).reset_index(drop=True)
                gdf_frame.insert(1, 'year', year)
                gdf_frame.insert(2, 'month', month)

                return gdf_frame

            except requests.RequestException as e:
                print(f"Error downloading the file: {e}")
            except Exception as e:
                print(f"An error occurred: {e}")
            finally:
                # Clean up the temporary file
                if 'temp_file_path' in locals() and os.path.exists(temp_file_path):
                    os.unlink(temp_file_path)
                    # print(f"Cleaned up: Temporary file removed.")

    else:
        print(f"No data found for {year}-{month:02d}")
        return None

In [4]:
def process_month(year, month, gdf_frames, threshold_snowcover):
    return get_snow_cover_data_filter(year, month, gdf_frames, threshold_snowcover)

In [5]:
gdf_frames = opera_utils.get_frame_geojson(as_geodataframe=True)
if gdf_frames.index.name == 'frame_id':     
    gdf_frames = gdf_frames.reset_index()

gdf_frames = gdf_frames.loc[gdf_frames.is_north_america.isin([1, '1'])].reset_index(drop=True) 

In [6]:
# Generate a list of (year, month) tuples for the date range
date_range = []
current_date = start_date
while current_date <= end_date:
    date_range.append((current_date.year, current_date.month))
    current_date += timedelta(days=32)
    current_date = current_date.replace(day=1)

# Process data for the entire date range in parallel with progress bar
all_gdf_frames = []
with ProcessPoolExecutor(max_workers=ncpus) as executor:
    futures = [executor.submit(process_month, year, month, gdf_frames, threshold_snowcover) for year, month in date_range]
    
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing months"):
        result = future.result()
        if result is not None:
            all_gdf_frames.append(result)

Processing months:   0%|          | 0/99 [00:00<?, ?it/s]

Streaming snow cover data for 2016-11...

Streaming snow cover data for 2016-08...

Streaming snow cover data for 2017-03...
Streaming snow cover data for 2016-07...
Streaming snow cover data for 2016-12...
Streaming snow cover data for 2017-04...
Streaming snow cover data for 2016-10...
Streaming snow cover data for 2017-01...
File downloaded successfully to temporary location: /tmp/tmpllc9ceb5.hdf
File downloaded successfully to temporary location: /tmp/tmp88013jxz.hdf
File downloaded successfully to temporary location: /tmp/tmpde4q3d9_.hdf
File downloaded successfully to temporary location: /tmp/tmpphiblj12.hdf
File downloaded successfully to temporary location: /tmp/tmp70ovuhqk.hdf
File downloaded successfully to temporary location: /tmp/tmpm63msvdy.hdf
File downloaded successfully to temporary location: /tmp/tmpgibxsiov.hdf
File downloaded successfully to temporary location: /tmp/tmpzf1ac4u8.hdf
File downloaded successfully to temporary location: /tmp/tmpb2_545i5.hdf
File download

Processing months:   1%|          | 1/99 [05:11<8:28:47, 311.50s/it]

Streaming snow cover data for 2017-05...
File downloaded successfully to temporary location: /tmp/tmpfgy31nxt.hdf


Processing months:   3%|▎         | 3/99 [05:19<1:55:30, 72.20s/it] 

...

Processing months:  89%|████████▉ | 88/99 [49:04<03:45, 20.53s/it]

Streaming snow cover data for 2024-08...
File downloaded successfully to temporary location: /tmp/tmpx5wuubnw.hdf


Processing months:  90%|████████▉ | 89/99 [49:18<03:06, 18.61s/it]

Streaming snow cover data for 2024-09...
File downloaded successfully to temporary location: /tmp/tmpyn26efmw.hdf


Processing months: 100%|██████████| 99/99 [54:40<00:00, 33.14s/it]


In [7]:
merged_gdf = pd.concat(all_gdf_frames, ignore_index=True)
merged_gdf = merged_gdf.sort_values(by=['year', 'month', 'frame_id']).reset_index(drop=True)

In [8]:
merged_gdf

,frame_id,year,month,geometry,to_process
0,95,2016,7,"POLYGON ((174.6763 52.5153, 172.89022 52.62514...",1
1,96,2016,7,"POLYGON ((174.30093 51.19519, 172.56574 51.304...",1
2,831,2016,7,"POLYGON ((-77.63462 33.64007, -77.94576 34.985...",1
3,832,2016,7,"POLYGON ((-77.94234 34.97052, -78.25801 36.313...",1
4,833,2016,7,"POLYGON ((-78.2545 36.29876, -78.57564 37.6411...",1
...,...,...,...,...,...
167673,46808,2024,9,"POLYGON ((-158.78991 58.35069, -160.86017 58.4...",1
167674,46809,2024,9,"POLYGON ((-159.23886 57.03479, -161.23659 57.1...",1
167675,46810,2024,9,"POLYGON ((-159.66635 55.71771, -161.59641 55.8...",1
167676,46811,2024,9,"POLYGON ((-159.36814 54.32632, -161.94392 54.5...",1


In [9]:
# Center coordinates of the US
us_center = [37.0902, -95.7129]
year_selected = 2016
month_selected = 12

# Plot to_process
merged_gdf[(merged_gdf['year'] == year_selected) & (merged_gdf['month'] == month_selected)][['frame_id', 'geometry', 'to_process']].explore(
    "to_process", 
    vmin=0,
    vmax=1,
    location=us_center, 
    zoom_start=3,
    cmap="RdBu",
)

In [10]:
year_selected = 2016
month_selected = 12
frameID_selected = 42779

# Plot to_process
merged_gdf[(merged_gdf['year'] == year_selected) & (merged_gdf['month'] == month_selected) & (merged_gdf['frame_id'] == frameID_selected)][['frame_id', 'geometry', 'to_process']].explore(
    "to_process", 
    vmin=0,
    vmax=1,
    cmap="RdBu",
)

In [11]:
year_selected = 2016
month_selected = 12
frameID_selected = 33065

# Plot to_process
merged_gdf[(merged_gdf['year'] == year_selected) & (merged_gdf['month'] == month_selected) & (merged_gdf['frame_id'] == frameID_selected)][['frame_id', 'geometry', 'to_process']].explore(
    "to_process", 
    vmin=0,
    vmax=1,
    cmap="RdBu",
)

In [12]:
# Export to geojson file
merged_gdf.to_file(output_geojson, driver="GeoJSON")